In [8]:
!pip install flask
!pip install pyngrok

In [9]:
!ngrok config add-authtoken YOUR-NGROK-API-HERE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
import getpass
import os
import threading
import requests

from flask import Flask, request, jsonify
from pyngrok import ngrok, conf

# Initialize Flask app
app = Flask(__name__)
port = "5000"

# Open an ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# OpenWeatherMap API (Replace 'your_api_key_here' with your actual API key)
WEATHER_API_KEY = "YOUR-OPENWEATHERMAP-API-HERE"
WEATHER_API_URL = "https://api.openweathermap.org/data/2.5/weather"

@app.route("/")
def index():
    return "Welcome to the Enhanced Flask App! Use the /weather endpoint to get live weather data."

@app.route("/weather", methods=["GET"])
def get_weather():
    """Fetch weather data for a given city"""
    city = request.args.get("city", default="London", type=str)
    params = {
        "q": city,
        "appid": WEATHER_API_KEY,
        "units": "metric"
    }
    response = requests.get(WEATHER_API_URL, params=params)

    if response.status_code == 200:
        weather_data = response.json()
        return jsonify({
            "city": weather_data["name"],
            "temperature": weather_data["main"]["temp"],
            "description": weather_data["weather"][0]["description"]
        })
    else:
        return jsonify({"error": "Unable to fetch weather data. Check the city name or API key."}), response.status_code

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


 * ngrok tunnel "https://d1ea-35-234-175-211.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
